In [ ]:
"""
Advanced Time Series Forecasting with Prophet and Optuna
File: prophet_optuna_pipeline.py

This script generates a synthetic multi-seasonal time series (3+ years daily),
implements walk-forward cross-validation, builds a baseline Prophet model,
optimizes hyperparameters using Optuna, trains a final optimized Prophet model,
compares it with an Exponential Smoothing baseline, and outputs forecast values
for the held-out test period.

Usage:
    python prophet_optuna_pipeline.py

Requirements:
    pip install prophet optuna statsmodels pandas numpy scikit-learn matplotlib

Notes:
- This code is intended for educational / research use. For production, adapt
  data-loading, logging, and model persistence to your environment.

"""

import warnings
warnings.filterwarnings("ignore")

import os
import math
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Prophet import with fallback
try:
    from prophet import Prophet
except Exception:
    try:
        from fbprophet import Prophet
    except Exception as e:
        raise ImportError(
            "Prophet is not installed. Install with `pip install prophet` or `pip install fbprophet`."
        )

import optuna
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt

# -----------------------------
# 1) DATA GENERATION
# -----------------------------

def generate_synthetic_daily(start_date="2016-01-01",
                             end_date="2020-12-31",
                             seed=42,
                             trend_slope=0.0007,
                             yearly_strength=30,
                             weekly_strength=7,
                             noise_std=3.0):
    """Generate synthetic daily time series with trend, yearly and weekly seasonalities.

    Parameters
    ----------
    start_date : str
        Start date (inclusive) in YYYY-MM-DD
    end_date : str
        End date (inclusive) in YYYY-MM-DD
    seed : int
        Random seed
    trend_slope : float
        Linear trend slope applied to the scaled time index
    yearly_strength : float
        Amplitude of yearly seasonality
    weekly_strength : float
        Amplitude of weekly seasonality
    noise_std : float
        Standard deviation of Gaussian noise

    Returns
    -------
    pd.DataFrame
        DataFrame with columns ['ds', 'y'] suitable for Prophet
    """
    np.random.seed(seed)
    idx = pd.date_range(start=start_date, end=end_date, freq='D')
    n = len(idx)

    t = np.arange(n) / n
    # Linear trend
    trend = trend_slope * np.arange(n)

    # Yearly seasonality (using a simple cosine with period ~365)
    day_of_year = idx.dayofyear - 1
    yearly = yearly_strength * np.cos(2 * np.pi * day_of_year / 365.25)

    # Weekly seasonality
    dow = idx.dayofweek
    weekly = weekly_strength * np.where(dow < 5, 1.0, 0.3)  # weekdays higher
    weekly = weekly * np.sin(2 * np.pi * dow / 7)

    # Add some occasional holiday spikes
    holiday_effect = np.zeros(n)
    # Synthetic holidays: pick ~6 random dates per year to add a bump
    years = np.unique(idx.year)
    for y in years:
        rng = np.random.default_rng(seed + int(y))
        picks = rng.choice(pd.date_range(start=f"{y}-01-01", end=f"{y}-12-31", freq='D').values,
                           size=6, replace=False)
        for p in picks:
            p = pd.to_datetime(p)
            loc = np.where(idx == p)[0]
            if len(loc) > 0:
                holiday_effect[loc[0]] += rng.normal(15, 5)

    noise = np.random.normal(0, noise_std, size=n)

    y = 100 + 10 * trend + yearly + weekly + holiday_effect + noise

    df = pd.DataFrame({
        'ds': idx,
        'y': y
    })
    return df


# -----------------------------
# 2) WALK-FORWARD CROSS-VALIDATION UTILITIES
# -----------------------------

def walk_forward_splits(df, initial_days=365 * 2, horizon_days=90, step_days=90):
    """Generate train/test splits for walk-forward (rolling-origin) CV.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with 'ds' sorted ascending.
    initial_days : int
        Size of the initial training window in days.
    horizon_days : int
        Forecast horizon (test) window length in days.
    step_days : int
        How much to move the window each iteration.

    Yields
    ------
    (train_df, test_df)
    """
    df = df.sort_values('ds').reset_index(drop=True)
    start = 0
    n = len(df)
    initial = initial_days
    while True:
        train_end = start + initial
        test_end = train_end + horizon_days
        if test_end > n:
            break
        train_df = df.iloc[:train_end].copy()
        test_df = df.iloc[train_end:test_end].copy()
        yield train_df, test_df
        start += step_days


# -----------------------------
# 3) METRICS
# -----------------------------

def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    # avoid division by zero
    denom = np.where(np.abs(y_true) < 1e-8, 1e-8, np.abs(y_true))
    return np.mean(np.abs((y_true - y_pred) / denom)) * 100.0


def evaluate_forecast(y_true, y_pred):
    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape_v = mape(y_true, y_pred)
    return {'RMSE': rmse, 'MAE': mae, 'MAPE': mape_v}


# -----------------------------
# 4) BASELINE PROPHET WITH DEFAULT PARAMETERS
# -----------------------------

def fit_prophet_and_forecast(train_df, periods, freq='D', prophet_kwargs=None):
    if prophet_kwargs is None:
        prophet_kwargs = {}
    m = Prophet(**prophet_kwargs)
    m.add_country_holidays(country_name='US')  # optional helpful prior
    m.fit(train_df)

    future = m.make_future_dataframe(periods=periods, freq=freq)
    forecast = m.predict(future)
    # keep only forecasted horizon
    return m, forecast


def baseline_prophet_cv(df, initial_days=365 * 2, horizon_days=90, step_days=90):
    """Run walk-forward CV with default Prophet and return aggregated metrics."""
    results = []
    for train_df, test_df in walk_forward_splits(df, initial_days, horizon_days, step_days):
        model, forecast = fit_prophet_and_forecast(train_df, periods=len(test_df))
        # extract forecasted values aligned with test_df
        y_pred = forecast.set_index('ds').loc[test_df['ds'], 'yhat'].values
        y_true = test_df['y'].values
        results.append(evaluate_forecast(y_true, y_pred))
    # aggregate
    df_res = pd.DataFrame(results)
    return df_res.mean().to_dict(), df_res


# -----------------------------
# 5) OPTUNA OPTIMIZATION
# -----------------------------

def prophet_objective_factory(df, initial_days=365 * 2, horizon_days=90, step_days=90, trials_per_split=1):
    """Return an Optuna objective function that performs walk-forward CV for a candidate set of hyperparameters.

    trials_per_split: normally 1. Kept for future extensions.
    """
    def objective(trial):
        # Suggest hyperparameters
        changepoint_prior_scale = trial.suggest_loguniform('changepoint_prior_scale', 0.001, 0.5)
        seasonality_prior_scale = trial.suggest_loguniform('seasonality_prior_scale', 0.01, 20.0)
        seasonality_mode = trial.suggest_categorical('seasonality_mode', ['additive', 'multiplicative'])
        n_changepoints = trial.suggest_int('n_changepoints', 5, 50)
        changepoint_range = trial.suggest_uniform('changepoint_range', 0.7, 0.95)

        prophet_kwargs = {
            'changepoint_prior_scale': changepoint_prior_scale,
            'seasonality_prior_scale': seasonality_prior_scale,
            'seasonality_mode': seasonality_mode,
            'n_changepoints': n_changepoints,
            'changepoint_range': changepoint_range
        }

        # perform walk-forward CV and get average RMSE across splits
        rmses = []
        for train_df, test_df in walk_forward_splits(df, initial_days, horizon_days, step_days):
            try:
                m = Prophet(**prophet_kwargs)
                m.add_country_holidays(country_name='US')
                m.fit(train_df)
                future = m.make_future_dataframe(periods=len(test_df), freq='D')
                forecast = m.predict(future)
                y_pred = forecast.set_index('ds').loc[test_df['ds'], 'yhat'].values
                y_true = test_df['y'].values
                rmses.append(math.sqrt(mean_squared_error(y_true, y_pred)))
            except Exception as e:
                # If training fails, return a large penalty
                return 1e6
        # objective to minimize
        return float(np.mean(rmses))

    return objective


def run_optuna(df, n_trials=40, direction='minimize', seed=42,
               initial_days=365 * 2, horizon_days=90, step_days=90):
    study = optuna.create_study(direction=direction, sampler=optuna.samplers.TPESampler(seed=seed))
    objective = prophet_objective_factory(df, initial_days, horizon_days, step_days)
    study.optimize(objective, n_trials=n_trials)
    return study


# -----------------------------
# 6) FINAL MODEL TRAINING & FORECAST
# -----------------------------

def train_final_prophet(df_train_full, best_params, periods):
    model = Prophet(**best_params)
    model.add_country_holidays(country_name='US')
    model.fit(df_train_full)
    future = model.make_future_dataframe(periods=periods, freq='D')
    forecast = model.predict(future)
    return model, forecast


# -----------------------------
# 7) COMPARISON BASELINE: EXPONENTIAL SMOOTHING
# -----------------------------

def exp_smoothing_forecast(train_series, periods):
    """Fit Holt-Winters Exponential Smoothing and forecast.

    train_series : pd.Series indexed by ds
    periods : int
    """
    model = ExponentialSmoothing(train_series, seasonal='add', seasonal_periods=365, trend='add')
    fit = model.fit(optimized=True)
    forecast = fit.forecast(periods)
    return forecast


# -----------------------------
# 8) MAIN EXECUTION
# -----------------------------


def main():
    # 1) Generate dataset
    df = generate_synthetic_daily(start_date='2016-01-01', end_date='2020-12-31')
    print(f"Generated dataset with {len(df)} daily rows from {df['ds'].min().date()} to {df['ds'].max().date()}")

    # 2) Split into train / held-out test
    # We'll hold out the last 180 days for final evaluation
    holdout_days = 180
    df_train_full = df.iloc[:-holdout_days].copy()
    df_holdout = df.iloc[-holdout_days:].copy()
    print(f"Train full length: {len(df_train_full)}, Holdout length: {len(df_holdout)}")

    # 3) Baseline Prophet CV
    print("Running baseline Prophet CV (this may take several minutes)...")
    baseline_mean_metrics, baseline_all = baseline_prophet_cv(df_train_full,
                                                             initial_days=365*2,
                                                             horizon_days=90,
                                                             step_days=90)
    print("Baseline CV average metrics:")
    print(baseline_mean_metrics)

    # 4) Run Optuna to optimize Prophet hyperparameters
    print("Running Optuna hyperparameter search (this may take several minutes)...")
    study = run_optuna(df_train_full, n_trials=40, seed=42,
                       initial_days=365*2, horizon_days=90, step_days=90)

    print("Best trial:")
    print(study.best_trial.params)
    best_params = study.best_trial.params

    # Map Optuna param names to Prophet kwargs (optuna returns floats; they are fine)
    prophet_best_kwargs = {
        'changepoint_prior_scale': best_params['changepoint_prior_scale'],
        'seasonality_prior_scale': best_params['seasonality_prior_scale'],
        'seasonality_mode': best_params['seasonality_mode'],
        'n_changepoints': int(best_params['n_changepoints']),
        'changepoint_range': best_params['changepoint_range']
    }

    # 5) Train final optimized Prophet on full train and forecast holdout
    print("Training final optimized Prophet on full training data...")
    model_opt, forecast_opt = train_final_prophet(df_train_full, prophet_best_kwargs, periods=holdout_days)

    # get forecast values corresponding to holdout ds
    forecast_values = forecast_opt.set_index('ds').loc[df_holdout['ds']]
    y_pred_opt = forecast_values['yhat'].values
    y_true = df_holdout['y'].values
    opt_metrics = evaluate_forecast(y_true, y_pred_opt)
    print("Optimized Prophet metrics on holdout:")
    print(opt_metrics)

    # 6) Baseline Exponential Smoothing forecast trained on df_train_full and evaluated on holdout
    print("Training Exponential Smoothing baseline and forecasting holdout...")
    train_series = df_train_full.set_index('ds')['y']
    es_forecast = exp_smoothing_forecast(train_series, periods=holdout_days)
    # align index
    es_forecast = pd.Series(es_forecast.values, index=df_holdout['ds'])
    es_metrics = evaluate_forecast(y_true, es_forecast.values)
    print("Exponential Smoothing metrics on holdout:")
    print(es_metrics)

    # 7) Compare CV baseline vs optimized CV
    # We'll also compute CV for the best params to compare CV performance
    print("Running CV for optimized parameters to compare with baseline CV (on training data)...")
    # small function to run CV for specific prophet kwargs
    def prophet_cv_with_kwargs(df_input, prophet_kwargs, initial_days=365*2, horizon_days=90, step_days=90):
        results = []
        for train_df, test_df in walk_forward_splits(df_input, initial_days, horizon_days, step_days):
            m = Prophet(**prophet_kwargs)
            m.add_country_holidays(country_name='US')
            m.fit(train_df)
            future = m.make_future_dataframe(periods=len(test_df), freq='D')
            forecast = m.predict(future)
            y_pred = forecast.set_index('ds').loc[test_df['ds'], 'yhat'].values
            y_true = test_df['y'].values
            results.append(evaluate_forecast(y_true, y_pred))
        df_res = pd.DataFrame(results)
        return df_res.mean().to_dict(), df_res

    opt_cv_mean_metrics, opt_cv_all = prophet_cv_with_kwargs(df_train_full, prophet_best_kwargs,
                                                              initial_days=365*2, horizon_days=90, step_days=90)
    print("Optimized CV average metrics (training data):")
    print(opt_cv_mean_metrics)

    # 8) Save final forecast values chronologically as text
    final_forecast_df = pd.DataFrame({
        'ds': df_holdout['ds'].values,
        'y_true': y_true,
        'yhat_optimized': y_pred_opt,
        'es_forecast': es_forecast.values
    })

    out_csv = 'final_holdout_forecasts.csv'
    final_forecast_df.to_csv(out_csv, index=False)
    print(f"Final holdout forecasts saved to {out_csv}")

    # 9) Print concise text-based report (summary)
    print("\n--- SUMMARY REPORT ---")
    print("Hyperparameter search space: changepoint_prior_scale [0.001,0.5] (log-uniform), seasonality_prior_scale [0.01,20] (log-uniform), seasonality_mode {additive,multiplicative}, n_changepoints [5,50] (int), changepoint_range [0.7,0.95] (uniform).")
    print(f"Best optuna params: {study.best_trial.params}")
    print("Baseline CV metrics (averaged across splits):")
    print(baseline_mean_metrics)
    print("Optimized CV metrics (averaged across splits):")
    print(opt_cv_mean_metrics)
    print("Holdout comparison on last {0} days:".format(holdout_days))
    print(f"Optimized Prophet on holdout: {opt_metrics}")
    print(f"Exponential Smoothing on holdout: {es_metrics}")

    # 10) Optional: save study results
    study_df = study.trials_dataframe()
    study_df.to_csv('optuna_study_trials.csv', index=False)
    print("Optuna trial details saved to optuna_study_trials.csv")

    # 11) Display the first 20 forecast rows as chronological text snippet
    print("\nFirst 20 rows of final holdout forecasts (chronological):")
    with pd.option_context('display.max_rows', 20, 'display.max_columns', None):
        print(final_forecast_df.head(20).to_string(index=False))

    # 12) Plot actual vs optimized forecast for holdout
    try:
        plt.figure(figsize=(12, 6))
        plt.plot(df['ds'], df['y'], label='history')
        plt.plot(final_forecast_df['ds'], final_forecast_df['yhat_optimized'], label='prophet_optimized_forecast')
        plt.plot(final_forecast_df['ds'], final_forecast_df['es_forecast'], label='exp_smoothing_forecast')
        plt.axvline(df_holdout['ds'].iloc[0], color='k', linestyle='--', alpha=0.6)
        plt.legend()
        plt.title('History and holdout forecasts')
        plt.xlabel('Date')
        plt.ylabel('y')
        plt.tight_layout()
        plt.savefig('forecast_comparison.png')
        print('Plot saved to forecast_comparison.png')
    except Exception as e:
        print('Could not create plot:', e)

    print('\nScript finished.')


if __name__ == '__main__':
    main()
